# 第53章: 3D座標変換と剛体運動

## 📋 この章で学ぶこと

この章を終えると、以下ができるようになります：

- [ ] 剛体変換（回転・並進）を理解し実装できる
- [ ] 回転の複数の表現方法を相互変換できる
- [ ] SE(3)群の基本的な性質を理解する
- [ ] 座標系間の変換を正しく適用できる

## 🎯 前提知識

- ✅ 線形代数（行列、固有値）
- ✅ Notebook 51（座標系の定義）

⏱️ **推定学習時間**: 150-180分  
📊 **難易度**: ★★★★☆（上級）  
🎓 **カテゴリ**: 理論 + 実践

---

## 🌟 はじめに

3D Computer Visionでは、異なる**座標系**間での変換が頻繁に必要になります：

- ワールド座標 → カメラ座標
- カメラ1の座標 → カメラ2の座標
- ロボットのベース → エンドエフェクタ

これらの変換は**剛体変換（Rigid Transformation）**で表され、距離と角度を保存します。

```
ワールド座標系              カメラ座標系
    Y                          Y'
    │                          │
    │   ●P                     │
    │  /                       │  ●P'
    │ /                        │ /
    │/                         │/
    ●───────X     [R|t]        ●────────X'
   /            ────────→     /
  Z                          Z'
```

この章では、剛体変換の数学的基礎と実装を学びます。

In [ ]:
# ============================================================
# 環境設定
# ============================================================

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import warnings

warnings.filterwarnings('ignore')

plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

# 日本語フォント設定
import matplotlib.font_manager as fm

def setup_japanese_font():
    japanese_fonts = [
        'Hiragino Sans', 'Hiragino Maru Gothic Pro', 'AppleGothic',
        'Yu Gothic', 'MS Gothic',
        'Noto Sans CJK JP', 'IPAexGothic', 'TakaoPGothic',
    ]
    available_fonts = set(f.name for f in fm.fontManager.ttflist)
    for font in japanese_fonts:
        if font in available_fonts:
            plt.rcParams['font.family'] = font
            plt.rcParams['axes.unicode_minus'] = False
            return font
    return None

font_used = setup_japanese_font()
if font_used:
    print(f"日本語フォント: {font_used}")

np.random.seed(42)

print("✅ ライブラリのインポート完了")

---

## 1. 剛体変換の基礎

### 1.1 剛体変換とは

**剛体変換（Rigid Body Transformation）**は、物体の形状を変えない変換です：

- **回転**（Rotation）: 向きを変える
- **並進**（Translation）: 位置を変える

**保存される性質:**
- 2点間の距離
- 角度
- 手の向き（右手系→右手系）

### 1.2 自由度（Degrees of Freedom）

3D空間での剛体変換は **6 DoF**（自由度）を持ちます：
- 回転: 3 DoF（各軸周りの回転）
- 並進: 3 DoF（各軸方向の移動）

In [ ]:
# ============================================================
# 6自由度の可視化
# ============================================================

fig = plt.figure(figsize=(16, 6))

# 左: 3つの回転
ax1 = fig.add_subplot(121, projection='3d')

# 座標軸
ax1.quiver(0, 0, 0, 1.5, 0, 0, color='red', arrow_length_ratio=0.1, linewidth=2)
ax1.quiver(0, 0, 0, 0, 1.5, 0, color='green', arrow_length_ratio=0.1, linewidth=2)
ax1.quiver(0, 0, 0, 0, 0, 1.5, color='blue', arrow_length_ratio=0.1, linewidth=2)

ax1.text(1.6, 0, 0, 'X (Roll)', fontsize=10, color='red')
ax1.text(0, 1.6, 0, 'Y (Pitch)', fontsize=10, color='green')
ax1.text(0, 0, 1.6, 'Z (Yaw)', fontsize=10, color='blue')

# 回転矢印（円弧）
theta = np.linspace(0, 1.5, 30)
r = 0.8

# Roll (X軸周り)
ax1.plot(np.zeros_like(theta), r*np.cos(theta), r*np.sin(theta), 'r--', linewidth=2)
ax1.annotate('', xy=(0, r*np.cos(1.5), r*np.sin(1.5)), xytext=(0, r*np.cos(1.3), r*np.sin(1.3)),
            arrowprops=dict(arrowstyle='->', color='red'))

# Pitch (Y軸周り)
ax1.plot(r*np.sin(theta), np.zeros_like(theta), r*np.cos(theta), 'g--', linewidth=2)

# Yaw (Z軸周り)
ax1.plot(r*np.cos(theta), r*np.sin(theta), np.zeros_like(theta), 'b--', linewidth=2)

ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_zlabel('Z')
ax1.set_title('回転 (3 DoF)\nRoll, Pitch, Yaw', fontsize=14, fontweight='bold')
ax1.set_xlim(-1.5, 1.5)
ax1.set_ylim(-1.5, 1.5)
ax1.set_zlim(-1.5, 1.5)

# 右: 3つの並進
ax2 = fig.add_subplot(122, projection='3d')

# 座標軸
ax2.quiver(0, 0, 0, 1.5, 0, 0, color='red', arrow_length_ratio=0.1, linewidth=2)
ax2.quiver(0, 0, 0, 0, 1.5, 0, color='green', arrow_length_ratio=0.1, linewidth=2)
ax2.quiver(0, 0, 0, 0, 0, 1.5, color='blue', arrow_length_ratio=0.1, linewidth=2)

# 並進矢印
ax2.quiver(0.5, 0.5, 0.5, 0.8, 0, 0, color='red', arrow_length_ratio=0.15, linewidth=3)
ax2.quiver(0.5, 0.5, 0.5, 0, 0.8, 0, color='green', arrow_length_ratio=0.15, linewidth=3)
ax2.quiver(0.5, 0.5, 0.5, 0, 0, 0.8, color='blue', arrow_length_ratio=0.15, linewidth=3)

ax2.text(1.5, 0.5, 0.5, 'tx', fontsize=12, color='red')
ax2.text(0.5, 1.5, 0.5, 'ty', fontsize=12, color='green')
ax2.text(0.5, 0.5, 1.5, 'tz', fontsize=12, color='blue')

# 原点の物体
ax2.scatter([0.5], [0.5], [0.5], s=100, c='black', marker='o')

ax2.set_xlabel('X')
ax2.set_ylabel('Y')
ax2.set_zlabel('Z')
ax2.set_title('並進 (3 DoF)\ntx, ty, tz', fontsize=14, fontweight='bold')
ax2.set_xlim(-0.5, 2)
ax2.set_ylim(-0.5, 2)
ax2.set_zlim(-0.5, 2)

plt.tight_layout()
plt.show()

print("💡 剛体変換 = 回転(3 DoF) + 並進(3 DoF) = 6 DoF")

---

## 2. 回転行列

### 2.1 SO(3)群

**回転行列**は特殊直交群 **SO(3)** に属します：

$$SO(3) = \{R \in \mathbb{R}^{3 \times 3} : R^T R = I, \det(R) = 1\}$$

**性質:**
- $R^T R = I$（直交性）
- $\det(R) = 1$（向きを保存）
- $R^{-1} = R^T$（逆行列が転置）

### 2.2 基本回転行列

各軸周りの回転行列：

In [ ]:
# ============================================================
# 基本回転行列の実装
# ============================================================

def rotation_matrix_x(theta):
    """
    X軸周りの回転行列 (Roll)
    
    Rx(θ) = | 1    0       0    |
            | 0  cos(θ) -sin(θ) |
            | 0  sin(θ)  cos(θ) |
    """
    c, s = np.cos(theta), np.sin(theta)
    return np.array([
        [1, 0,  0],
        [0, c, -s],
        [0, s,  c]
    ])


def rotation_matrix_y(theta):
    """
    Y軸周りの回転行列 (Pitch)
    
    Ry(θ) = |  cos(θ)  0  sin(θ) |
            |    0     1    0    |
            | -sin(θ)  0  cos(θ) |
    """
    c, s = np.cos(theta), np.sin(theta)
    return np.array([
        [ c, 0, s],
        [ 0, 1, 0],
        [-s, 0, c]
    ])


def rotation_matrix_z(theta):
    """
    Z軸周りの回転行列 (Yaw)
    
    Rz(θ) = | cos(θ) -sin(θ)  0 |
            | sin(θ)  cos(θ)  0 |
            |   0       0     1 |
    """
    c, s = np.cos(theta), np.sin(theta)
    return np.array([
        [c, -s, 0],
        [s,  c, 0],
        [0,  0, 1]
    ])


# 検証: SO(3)の性質
theta = np.radians(45)
R = rotation_matrix_z(theta)

print("Z軸周り45°回転行列:")
print(R)
print()
print(f"R^T @ R = I ?")
print(np.round(R.T @ R, 10))
print()
print(f"det(R) = {np.linalg.det(R):.6f} (should be 1)")

In [ ]:
# ============================================================
# 回転行列の効果を可視化
# ============================================================

def plot_rotation_effect(ax, R, title):
    """回転行列の効果を可視化"""
    # 元の座標軸
    axes_orig = np.eye(3)
    colors = ['red', 'green', 'blue']
    labels = ['X', 'Y', 'Z']
    
    # 回転後の座標軸
    axes_rot = R @ axes_orig
    
    for i in range(3):
        # 元の軸（薄い色）
        ax.quiver(0, 0, 0, axes_orig[0, i], axes_orig[1, i], axes_orig[2, i],
                  color=colors[i], alpha=0.3, arrow_length_ratio=0.1, linewidth=2)
        # 回転後の軸（濃い色）
        ax.quiver(0, 0, 0, axes_rot[0, i], axes_rot[1, i], axes_rot[2, i],
                  color=colors[i], arrow_length_ratio=0.1, linewidth=3)
        ax.text(axes_rot[0, i]*1.1, axes_rot[1, i]*1.1, axes_rot[2, i]*1.1,
                labels[i] + "'", fontsize=10, color=colors[i])
    
    ax.set_xlim(-1.5, 1.5)
    ax.set_ylim(-1.5, 1.5)
    ax.set_zlim(-1.5, 1.5)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title(title, fontsize=12, fontweight='bold')


fig = plt.figure(figsize=(18, 5))

angle = np.radians(45)

rotations = [
    (rotation_matrix_x(angle), 'Rx(45°): X軸周り'),
    (rotation_matrix_y(angle), 'Ry(45°): Y軸周り'),
    (rotation_matrix_z(angle), 'Rz(45°): Z軸周り'),
]

for i, (R, title) in enumerate(rotations):
    ax = fig.add_subplot(1, 3, i+1, projection='3d')
    plot_rotation_effect(ax, R, title)

plt.tight_layout()
plt.show()

print("💡 薄い軸が元の座標系、濃い軸が回転後の座標系")

---

## 3. 回転の表現方法

### 3.1 表現方法の比較

| 表現 | パラメータ数 | 特徴 | 用途 |
|------|------------|------|------|
| **回転行列** | 9 (実質3) | 直感的、冗長 | 変換の適用 |
| **オイラー角** | 3 | 人間に分かりやすい | UI、表示 |
| **軸角度** | 4 (3+1) | 幾何学的に明確 | 理解、説明 |
| **ロドリゲス** | 3 | コンパクト | 最適化 |
| **クォータニオン** | 4 | 補間に最適、特異点なし | アニメーション |

### 3.2 オイラー角

3つの角度で回転を表現します。回転の**順序**が重要です。

**ZYX順序（Yaw-Pitch-Roll）:**
$$R = R_z(\text{yaw}) \cdot R_y(\text{pitch}) \cdot R_x(\text{roll})$$

**⚠️ ジンバルロック問題**

Pitchが±90°に近づくと、YawとRollが区別できなくなる現象です。

In [ ]:
# ============================================================
# オイラー角と回転行列の変換
# ============================================================

def euler_to_rotation_matrix(roll, pitch, yaw, order='zyx'):
    """
    オイラー角から回転行列を生成
    
    Parameters:
    -----------
    roll : float
        X軸周りの回転（ラジアン）
    pitch : float
        Y軸周りの回転（ラジアン）
    yaw : float
        Z軸周りの回転（ラジアン）
    order : str
        回転順序（'xyz', 'zyx', etc.）
    
    Returns:
    --------
    R : np.ndarray (3, 3)
        回転行列
    """
    Rx = rotation_matrix_x(roll)
    Ry = rotation_matrix_y(pitch)
    Rz = rotation_matrix_z(yaw)
    
    rotation_map = {'x': Rx, 'y': Ry, 'z': Rz}
    
    R = np.eye(3)
    for axis in order:
        R = rotation_map[axis] @ R
    
    return R


def rotation_matrix_to_euler(R, order='zyx'):
    """
    回転行列からオイラー角を抽出（ZYX順序）
    
    注意: ジンバルロック時は一意に決まらない
    """
    if order == 'zyx':
        # pitch (Y軸)
        sy = -R[2, 0]
        
        if abs(sy) >= 1 - 1e-6:  # ジンバルロック
            pitch = np.sign(sy) * np.pi / 2
            roll = 0
            yaw = np.arctan2(-R[0, 1], R[1, 1])
        else:
            pitch = np.arcsin(sy)
            roll = np.arctan2(R[2, 1], R[2, 2])
            yaw = np.arctan2(R[1, 0], R[0, 0])
        
        return roll, pitch, yaw
    else:
        raise NotImplementedError(f"Order {order} not implemented")


# テスト
roll = np.radians(30)
pitch = np.radians(45)
yaw = np.radians(60)

R = euler_to_rotation_matrix(roll, pitch, yaw, order='zyx')
roll_back, pitch_back, yaw_back = rotation_matrix_to_euler(R, order='zyx')

print("オイラー角 → 回転行列 → オイラー角")
print(f"元の角度: roll={np.degrees(roll):.1f}°, pitch={np.degrees(pitch):.1f}°, yaw={np.degrees(yaw):.1f}°")
print(f"復元角度: roll={np.degrees(roll_back):.1f}°, pitch={np.degrees(pitch_back):.1f}°, yaw={np.degrees(yaw_back):.1f}°")

In [ ]:
# ============================================================
# ジンバルロックの可視化
# ============================================================

fig = plt.figure(figsize=(16, 5))

# 正常なケース
ax1 = fig.add_subplot(131, projection='3d')
R_normal = euler_to_rotation_matrix(np.radians(30), np.radians(30), np.radians(30))
plot_rotation_effect(ax1, R_normal, 'Pitch=30° (正常)')

# ジンバルロック付近
ax2 = fig.add_subplot(132, projection='3d')
R_near_lock = euler_to_rotation_matrix(np.radians(30), np.radians(85), np.radians(30))
plot_rotation_effect(ax2, R_near_lock, 'Pitch=85° (ジンバルロック付近)')

# ジンバルロック
ax3 = fig.add_subplot(133, projection='3d')
R_lock = euler_to_rotation_matrix(np.radians(30), np.radians(90), np.radians(30))
plot_rotation_effect(ax3, R_lock, 'Pitch=90° (ジンバルロック)')

plt.tight_layout()
plt.show()

print("💡 ジンバルロック: Pitch=±90°のとき、RollとYawが区別できなくなる")
print("   この問題を避けるため、クォータニオンが使われます")

### 3.3 軸角度表現とロドリゲスの公式

任意の回転は、ある**軸** $\mathbf{k}$ の周りを**角度** $\theta$ だけ回転することで表せます。

**ロドリゲスの公式:**

$$R = I + \sin(\theta)[\mathbf{k}]_\times + (1-\cos(\theta))[\mathbf{k}]_\times^2$$

ここで $[\mathbf{k}]_\times$ は**歪対称行列**（外積を行列で表したもの）：

$$[\mathbf{k}]_\times = \begin{bmatrix} 0 & -k_z & k_y \\ k_z & 0 & -k_x \\ -k_y & k_x & 0 \end{bmatrix}$$

In [ ]:
# ============================================================
# ロドリゲスの公式
# ============================================================

def skew_symmetric(v):
    """
    ベクトルから歪対称行列を生成
    
    [v]× = |  0   -vz   vy |
           |  vz   0   -vx |
           | -vy   vx   0  |
    """
    v = np.asarray(v).flatten()
    return np.array([
        [0, -v[2], v[1]],
        [v[2], 0, -v[0]],
        [-v[1], v[0], 0]
    ])


def rodrigues(axis, theta):
    """
    ロドリゲスの公式による回転行列
    
    R = I + sin(θ)[k]× + (1-cos(θ))[k]×²
    
    Parameters:
    -----------
    axis : array (3,)
        回転軸（正規化される）
    theta : float
        回転角度（ラジアン）
    
    Returns:
    --------
    R : np.ndarray (3, 3)
        回転行列
    """
    axis = np.asarray(axis).flatten()
    axis = axis / np.linalg.norm(axis)  # 正規化
    
    K = skew_symmetric(axis)
    I = np.eye(3)
    
    R = I + np.sin(theta) * K + (1 - np.cos(theta)) * (K @ K)
    return R


def rodrigues_vector(rvec):
    """
    回転ベクトルから回転行列を生成
    
    回転ベクトル: rvec = θ * k （ノルムが角度、方向が軸）
    """
    rvec = np.asarray(rvec).flatten()
    theta = np.linalg.norm(rvec)
    
    if theta < 1e-10:
        return np.eye(3)
    
    axis = rvec / theta
    return rodrigues(axis, theta)


# 検証: Z軸周り45度回転
axis = np.array([0, 0, 1])  # Z軸
theta = np.radians(45)

R_rodrigues = rodrigues(axis, theta)
R_direct = rotation_matrix_z(theta)

print("ロドリゲスの公式 vs 直接計算")
print(f"差のノルム: {np.linalg.norm(R_rodrigues - R_direct):.2e}")
print()
print("回転行列 (ロドリゲス):")
print(np.round(R_rodrigues, 4))

### 3.4 クォータニオン

**クォータニオン（Quaternion）**は4つの数で回転を表現します：

$$q = w + xi + yj + zk = (w, \mathbf{v})$$

軸 $\mathbf{k}$、角度 $\theta$ の回転は：

$$q = \left(\cos\frac{\theta}{2}, \sin\frac{\theta}{2} \cdot \mathbf{k}\right)$$

**利点:**
- ジンバルロックがない
- 滑らかな補間（SLERP）が可能
- 計算効率が良い

In [ ]:
# ============================================================
# クォータニオンの実装
# ============================================================

class Quaternion:
    """クォータニオンクラス"""
    
    def __init__(self, w, x, y, z):
        self.w = w
        self.x = x
        self.y = y
        self.z = z
    
    @classmethod
    def from_axis_angle(cls, axis, theta):
        """軸角度表現からクォータニオンを生成"""
        axis = np.asarray(axis).flatten()
        axis = axis / np.linalg.norm(axis)
        
        w = np.cos(theta / 2)
        v = np.sin(theta / 2) * axis
        
        return cls(w, v[0], v[1], v[2])
    
    @classmethod
    def from_rotation_matrix(cls, R):
        """回転行列からクォータニオンを生成"""
        trace = np.trace(R)
        
        if trace > 0:
            s = 0.5 / np.sqrt(trace + 1.0)
            w = 0.25 / s
            x = (R[2, 1] - R[1, 2]) * s
            y = (R[0, 2] - R[2, 0]) * s
            z = (R[1, 0] - R[0, 1]) * s
        elif R[0, 0] > R[1, 1] and R[0, 0] > R[2, 2]:
            s = 2.0 * np.sqrt(1.0 + R[0, 0] - R[1, 1] - R[2, 2])
            w = (R[2, 1] - R[1, 2]) / s
            x = 0.25 * s
            y = (R[0, 1] + R[1, 0]) / s
            z = (R[0, 2] + R[2, 0]) / s
        elif R[1, 1] > R[2, 2]:
            s = 2.0 * np.sqrt(1.0 + R[1, 1] - R[0, 0] - R[2, 2])
            w = (R[0, 2] - R[2, 0]) / s
            x = (R[0, 1] + R[1, 0]) / s
            y = 0.25 * s
            z = (R[1, 2] + R[2, 1]) / s
        else:
            s = 2.0 * np.sqrt(1.0 + R[2, 2] - R[0, 0] - R[1, 1])
            w = (R[1, 0] - R[0, 1]) / s
            x = (R[0, 2] + R[2, 0]) / s
            y = (R[1, 2] + R[2, 1]) / s
            z = 0.25 * s
        
        return cls(w, x, y, z)
    
    def to_rotation_matrix(self):
        """回転行列に変換"""
        w, x, y, z = self.w, self.x, self.y, self.z
        
        return np.array([
            [1-2*(y*y+z*z), 2*(x*y-z*w), 2*(x*z+y*w)],
            [2*(x*y+z*w), 1-2*(x*x+z*z), 2*(y*z-x*w)],
            [2*(x*z-y*w), 2*(y*z+x*w), 1-2*(x*x+y*y)]
        ])
    
    def normalize(self):
        """正規化"""
        norm = np.sqrt(self.w**2 + self.x**2 + self.y**2 + self.z**2)
        return Quaternion(self.w/norm, self.x/norm, self.y/norm, self.z/norm)
    
    def conjugate(self):
        """共役クォータニオン"""
        return Quaternion(self.w, -self.x, -self.y, -self.z)
    
    def __mul__(self, other):
        """クォータニオン積"""
        w1, x1, y1, z1 = self.w, self.x, self.y, self.z
        w2, x2, y2, z2 = other.w, other.x, other.y, other.z
        
        return Quaternion(
            w1*w2 - x1*x2 - y1*y2 - z1*z2,
            w1*x2 + x1*w2 + y1*z2 - z1*y2,
            w1*y2 - x1*z2 + y1*w2 + z1*x2,
            w1*z2 + x1*y2 - y1*x2 + z1*w2
        )
    
    def __repr__(self):
        return f"Quaternion({self.w:.4f}, {self.x:.4f}, {self.y:.4f}, {self.z:.4f})"


# テスト
axis = np.array([0, 0, 1])  # Z軸
theta = np.radians(90)

q = Quaternion.from_axis_angle(axis, theta)
R_from_q = q.to_rotation_matrix()
R_direct = rotation_matrix_z(theta)

print(f"クォータニオン: {q}")
print(f"回転行列への変換誤差: {np.linalg.norm(R_from_q - R_direct):.2e}")

In [ ]:
# ============================================================
# SLERP（球面線形補間）
# ============================================================

def slerp(q1, q2, t):
    """
    球面線形補間（Spherical Linear Interpolation）
    
    Parameters:
    -----------
    q1, q2 : Quaternion
        始点と終点のクォータニオン
    t : float
        補間パラメータ [0, 1]
    
    Returns:
    --------
    q : Quaternion
        補間されたクォータニオン
    """
    # 内積
    dot = q1.w*q2.w + q1.x*q2.x + q1.y*q2.y + q1.z*q2.z
    
    # 最短経路を取るため、必要に応じて符号反転
    if dot < 0:
        q2 = Quaternion(-q2.w, -q2.x, -q2.y, -q2.z)
        dot = -dot
    
    # ほぼ同じ方向の場合は線形補間
    if dot > 0.9995:
        result = Quaternion(
            q1.w + t * (q2.w - q1.w),
            q1.x + t * (q2.x - q1.x),
            q1.y + t * (q2.y - q1.y),
            q1.z + t * (q2.z - q1.z)
        )
        return result.normalize()
    
    # 球面補間
    theta_0 = np.arccos(dot)
    theta = theta_0 * t
    
    sin_theta = np.sin(theta)
    sin_theta_0 = np.sin(theta_0)
    
    s1 = np.cos(theta) - dot * sin_theta / sin_theta_0
    s2 = sin_theta / sin_theta_0
    
    return Quaternion(
        s1 * q1.w + s2 * q2.w,
        s1 * q1.x + s2 * q2.x,
        s1 * q1.y + s2 * q2.y,
        s1 * q1.z + s2 * q2.z
    )


# SLERPの可視化
q1 = Quaternion.from_axis_angle([0, 0, 1], np.radians(0))
q2 = Quaternion.from_axis_angle([0, 0, 1], np.radians(90))

fig = plt.figure(figsize=(16, 5))

t_values = [0, 0.25, 0.5, 0.75, 1.0]

for i, t in enumerate(t_values):
    ax = fig.add_subplot(1, 5, i+1, projection='3d')
    
    q_interp = slerp(q1, q2, t)
    R = q_interp.to_rotation_matrix()
    
    plot_rotation_effect(ax, R, f't = {t}')

plt.suptitle('SLERP補間 (Z軸周り 0° → 90°)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("💡 SLERPは一定の角速度で滑らかに回転を補間します")

---

## 4. 同次変換行列とSE(3)

### 4.1 同次座標

3D点 $(x, y, z)$ を**同次座標**で表現：

$$\mathbf{X}_h = \begin{bmatrix} x \\ y \\ z \\ 1 \end{bmatrix}$$

これにより、回転と並進を**1つの行列**で表現できます。

### 4.2 同次変換行列

$$T = \begin{bmatrix} R & \mathbf{t} \\ \mathbf{0}^T & 1 \end{bmatrix} = \begin{bmatrix} r_{11} & r_{12} & r_{13} & t_x \\ r_{21} & r_{22} & r_{23} & t_y \\ r_{31} & r_{32} & r_{33} & t_z \\ 0 & 0 & 0 & 1 \end{bmatrix}$$

変換の適用:
$$\mathbf{X}'_h = T \cdot \mathbf{X}_h$$

In [ ]:
# ============================================================
# 同次変換行列
# ============================================================

def create_transformation_matrix(R, t):
    """
    回転行列と並進ベクトルから4x4変換行列を生成
    
    T = | R  t |
        | 0  1 |
    """
    t = np.asarray(t).flatten()
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = t
    return T


def decompose_transformation_matrix(T):
    """変換行列から回転行列と並進ベクトルを抽出"""
    R = T[:3, :3]
    t = T[:3, 3]
    return R, t


def inverse_transformation(T):
    """
    変換行列の逆行列
    
    T^{-1} = | R^T  -R^T t |
             | 0       1   |
    """
    R, t = decompose_transformation_matrix(T)
    R_inv = R.T
    t_inv = -R_inv @ t
    return create_transformation_matrix(R_inv, t_inv)


def transform_points(T, points):
    """
    点群を変換
    
    Parameters:
    -----------
    T : np.ndarray (4, 4)
        変換行列
    points : np.ndarray (N, 3)
        点群
    
    Returns:
    --------
    transformed : np.ndarray (N, 3)
        変換後の点群
    """
    points = np.asarray(points)
    if points.ndim == 1:
        points = points.reshape(1, -1)
    
    # 同次座標に変換
    ones = np.ones((points.shape[0], 1))
    points_h = np.hstack([points, ones])  # (N, 4)
    
    # 変換
    transformed_h = (T @ points_h.T).T  # (N, 4)
    
    return transformed_h[:, :3]


# テスト
R = rotation_matrix_z(np.radians(45))
t = np.array([1, 2, 0])

T = create_transformation_matrix(R, t)

print("同次変換行列 T:")
print(np.round(T, 4))
print()

# 点を変換
point = np.array([1, 0, 0])
point_transformed = transform_points(T, point)[0]

print(f"元の点: {point}")
print(f"変換後: {np.round(point_transformed, 4)}")
print()

# 逆変換の検証
T_inv = inverse_transformation(T)
point_back = transform_points(T_inv, point_transformed)[0]
print(f"逆変換で復元: {np.round(point_back, 4)}")

In [ ]:
# ============================================================
# 変換の合成
# ============================================================

fig = plt.figure(figsize=(16, 6))

# 元の立方体
def create_cube_vertices(center, size=1):
    """立方体の頂点を生成"""
    half = size / 2
    offsets = np.array([
        [-1, -1, -1], [1, -1, -1], [1, 1, -1], [-1, 1, -1],
        [-1, -1,  1], [1, -1,  1], [1, 1,  1], [-1, 1,  1]
    ]) * half
    return center + offsets

def plot_cube(ax, vertices, color='blue', alpha=0.3, label=None):
    """立方体を描画"""
    edges = [
        (0, 1), (1, 2), (2, 3), (3, 0),
        (4, 5), (5, 6), (6, 7), (7, 4),
        (0, 4), (1, 5), (2, 6), (3, 7)
    ]
    for i, j in edges:
        ax.plot([vertices[i, 0], vertices[j, 0]],
                [vertices[i, 1], vertices[j, 1]],
                [vertices[i, 2], vertices[j, 2]], color=color, linewidth=2, alpha=alpha)
    if label:
        ax.scatter(vertices[:, 0].mean(), vertices[:, 1].mean(), vertices[:, 2].mean(),
                  s=100, color=color, label=label)


# 変換を定義
# T1: 45度回転
R1 = rotation_matrix_z(np.radians(45))
t1 = np.array([0, 0, 0])
T1 = create_transformation_matrix(R1, t1)

# T2: 平行移動
R2 = np.eye(3)
t2 = np.array([2, 1, 0])
T2 = create_transformation_matrix(R2, t2)

# 合成: T_combined = T2 @ T1 (まずT1、次にT2)
T_combined = T2 @ T1

# 立方体
cube_orig = create_cube_vertices(np.array([0, 0, 0]), size=1)

ax = fig.add_subplot(121, projection='3d')

# 元の立方体
plot_cube(ax, cube_orig, color='blue', alpha=0.8, label='元の立方体')

# T1後
cube_after_t1 = transform_points(T1, cube_orig)
plot_cube(ax, cube_after_t1, color='green', alpha=0.5, label='T1後（回転）')

# T2後（T1の後にT2）
cube_after_t2 = transform_points(T2, cube_after_t1)
plot_cube(ax, cube_after_t2, color='red', alpha=0.5, label='T2後（平行移動）')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('順次変換: T2 @ T1', fontsize=12, fontweight='bold')
ax.legend()
ax.set_xlim(-2, 4)
ax.set_ylim(-2, 4)
ax.set_zlim(-2, 2)

# 右: 合成変換
ax = fig.add_subplot(122, projection='3d')

plot_cube(ax, cube_orig, color='blue', alpha=0.8, label='元の立方体')
cube_combined = transform_points(T_combined, cube_orig)
plot_cube(ax, cube_combined, color='red', alpha=0.5, label='合成変換')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('合成変換: T_combined = T2 @ T1', fontsize=12, fontweight='bold')
ax.legend()
ax.set_xlim(-2, 4)
ax.set_ylim(-2, 4)
ax.set_zlim(-2, 2)

plt.tight_layout()
plt.show()

print("💡 変換の合成: T_combined = T2 @ T1 (右から左に適用)")

---

## 5. 座標系間の変換

### 5.1 ワールド座標系 ↔ カメラ座標系

カメラの**外部パラメータ**は、ワールド座標系からカメラ座標系への変換を表します：

$$\mathbf{P}_{cam} = [R | \mathbf{t}] \mathbf{P}_{world}$$

ここで：
- $R$: カメラの向き（回転行列）
- $\mathbf{t}$: カメラ中心のワールド座標（の逆）

In [ ]:
# ============================================================
# ワールド座標系とカメラ座標系
# ============================================================

def look_at(camera_position, target, up=np.array([0, 1, 0])):
    """
    カメラ位置とターゲットからカメラポーズを生成（look-at変換）
    
    Parameters:
    -----------
    camera_position : array (3,)
        カメラの位置（ワールド座標）
    target : array (3,)
        注視点（ワールド座標）
    up : array (3,)
        上方向ベクトル
    
    Returns:
    --------
    T_w2c : np.ndarray (4, 4)
        ワールド→カメラ変換行列
    """
    camera_position = np.asarray(camera_position).flatten()
    target = np.asarray(target).flatten()
    up = np.asarray(up).flatten()
    
    # カメラのZ軸（視線方向の逆）
    z = camera_position - target
    z = z / np.linalg.norm(z)
    
    # カメラのX軸
    x = np.cross(up, z)
    x = x / np.linalg.norm(x)
    
    # カメラのY軸
    y = np.cross(z, x)
    
    # 回転行列（ワールド→カメラ）
    R = np.stack([x, y, z], axis=0)
    
    # 並進ベクトル
    t = -R @ camera_position
    
    return create_transformation_matrix(R, t)


# カメラのセットアップ
camera_pos = np.array([3, 2, 2])
target = np.array([0, 0, 0])

T_w2c = look_at(camera_pos, target)
T_c2w = inverse_transformation(T_w2c)

print("ワールド→カメラ変換行列:")
print(np.round(T_w2c, 4))
print()

# カメラ中心をワールド座標で確認
camera_center_world = transform_points(T_c2w, np.array([0, 0, 0]))[0]
print(f"カメラ中心（ワールド座標）: {np.round(camera_center_world, 4)}")
print(f"設定値: {camera_pos}")

In [ ]:
# ============================================================
# 座標系の可視化
# ============================================================

def plot_coordinate_frame(ax, T, scale=1.0, label='', colors=['red', 'green', 'blue']):
    """座標系を描画"""
    # 原点
    origin = transform_points(T, np.array([0, 0, 0]))[0]
    
    # 軸
    axes = np.eye(3) * scale
    axes_transformed = transform_points(T, axes)
    
    for i, (axis, color, name) in enumerate(zip(axes_transformed, colors, ['X', 'Y', 'Z'])):
        ax.quiver(origin[0], origin[1], origin[2],
                  axis[0] - origin[0], axis[1] - origin[1], axis[2] - origin[2],
                  color=color, arrow_length_ratio=0.1, linewidth=2)
    
    if label:
        ax.text(origin[0], origin[1], origin[2] + 0.3, label, fontsize=10)


fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

# ワールド座標系
plot_coordinate_frame(ax, np.eye(4), scale=1.5, label='World')

# カメラ座標系
plot_coordinate_frame(ax, T_c2w, scale=1.0, label='Camera', colors=['darkred', 'darkgreen', 'darkblue'])

# ターゲット点
ax.scatter([target[0]], [target[1]], [target[2]], s=100, c='yellow', marker='*', label='Target')

# カメラから視線方向を描画
look_dir = target - camera_pos
look_dir = look_dir / np.linalg.norm(look_dir) * 2
ax.quiver(camera_pos[0], camera_pos[1], camera_pos[2],
          look_dir[0], look_dir[1], look_dir[2],
          color='purple', arrow_length_ratio=0.05, linewidth=2, linestyle='--', alpha=0.5)

# オブジェクト（立方体）
cube = create_cube_vertices(np.array([0, 0, 0]), size=0.5)
plot_cube(ax, cube, color='gray', alpha=0.5)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('ワールド座標系とカメラ座標系', fontsize=14, fontweight='bold')
ax.legend()
ax.set_xlim(-2, 4)
ax.set_ylim(-2, 4)
ax.set_zlim(-1, 3)

plt.tight_layout()
plt.show()

---

## まとめ

### 🎯 このノートブックで学んだこと

**剛体変換**
- ✓ 回転（3 DoF）+ 並進（3 DoF）= 6 DoF
- ✓ 距離と角度を保存

**回転の表現**
- ✓ 回転行列: $R \in SO(3)$、$R^T R = I$、$\det(R) = 1$
- ✓ オイラー角: Roll-Pitch-Yaw、ジンバルロック問題
- ✓ ロドリゲス: $R = I + \sin\theta[k]_\times + (1-\cos\theta)[k]_\times^2$
- ✓ クォータニオン: SLERP補間、特異点なし

**同次変換**
- ✓ 4x4行列で回転と並進を統一的に扱う
- ✓ 変換の合成: $T_{combined} = T_2 \cdot T_1$

### 📊 表現方法のまとめ

| 表現 | パラメータ | メリット | デメリット |
|------|-----------|----------|------------|
| 回転行列 | 9 | 変換が容易 | 冗長、正規化必要 |
| オイラー角 | 3 | 直感的 | ジンバルロック |
| 軸角度 | 4 | 幾何学的 | 合成が複雑 |
| クォータニオン | 4 | 補間、安定 | 直感的でない |

---

## 🎓 自己評価クイズ

### Q1: SO(3)群の定義を説明してください

<details>
<summary>💡 答えを見る</summary>

**SO(3)** = Special Orthogonal group in 3D

$$SO(3) = \{R \in \mathbb{R}^{3 \times 3} : R^T R = I, \det(R) = 1\}$$

- $R^T R = I$: 直交行列（長さと角度を保存）
- $\det(R) = 1$: 向きを保存（鏡映を含まない）

</details>

---

### Q2: ジンバルロックとは何ですか？どう回避しますか？

<details>
<summary>💡 答えを見る</summary>

**ジンバルロック**: オイラー角表現で、Pitchが±90°のとき、RollとYawが区別できなくなる現象。

**回避方法**:
1. クォータニオンを使用する
2. 回転行列を直接使用する
3. オイラー角を使う場合は、特異点を避ける設計をする

</details>

---

### Q3: 変換 T1 の後に T2 を適用する合成変換は？

<details>
<summary>💡 答えを見る</summary>

$$T_{combined} = T_2 \cdot T_1$$

行列の積は右から左に適用されます：
$$\mathbf{P}' = T_2 \cdot T_1 \cdot \mathbf{P} = T_2 \cdot (T_1 \cdot \mathbf{P})$$

</details>

---

### ✅ 学習チェックリスト

- [ ] 基本回転行列（Rx, Ry, Rz）を書ける
- [ ] オイラー角から回転行列を計算できる
- [ ] ロドリゲスの公式を説明できる
- [ ] クォータニオンの基本操作ができる
- [ ] 同次変換行列の合成と逆変換ができる

---

**次のステップ**: Notebook 54で、**カメラキャリブレーション**を学びます！

実際のカメラから内部・外部パラメータを推定する方法を学びます。